In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data = keras.utils.image_dataset_from_directory(
    directory='dataset/Training',
    labels='inferred',
    label_mode='categorical',
    batch_size = 10,
    image_size=(256,256)
)
test_data = keras.utils.image_dataset_from_directory(
    directory='dataset/Testing',
    labels='inferred',
    label_mode='categorical',
    batch_size = 10,
    image_size=(256,256)
)

In [ ]:
train_data

In [2]:
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout,Activation,MaxPooling2D
from keras import Sequential

In [3]:
model = Sequential()

In [ ]:
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(128,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_data,epochs=10,validation_data=test_data)

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices('GPU')

In [3]:
from keras.applications.vgg16 import VGG16

In [4]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [5]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [6]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16,878,916
Trainable params: 16,878,916
No

In [8]:
conv_base.trainable = False

In [9]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

classes = os.listdir("dataset/Training")

X = []
y = []
image_size = 150
for i in classes:
    folderPath = os.path.join('dataset/Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X.append(img)
        y.append(i)
        
for i in classes:
    folderPath = os.path.join('dataset/Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X.append(img)
        y.append(i)
        
X = np.array(X)
y = np.array(y)

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=42)

100%|█████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 277.45it/s]


In [11]:

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
datagen.fit(X_train)
datagen.fit(X_test)

y_train_new = []
for i in y_train:
    y_train_new.append(classes.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(classes.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [22]:
model.fit(datagen.flow(X_train, y_train, batch_size=32,
         subset='training'),
         validation_data=datagen.flow(X_train, y_train,
         batch_size=8, subset='validation'),
         steps_per_epoch=len(X_train) / 32, epochs=10)

Epoch 1/10
81/81 [==============================] - 13s 158ms/step - loss: 0.7359 - accuracy: 0.7190 - val_loss: 0.6058 - val_accuracy: 0.8008


In [12]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'dataset/Training',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'dataset/Testing',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [ ]:
model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator
)

Epoch 1/10
90/90 [==============================] - 13s 139ms/step - loss: 0.1704 - accuracy: 0.9300 - val_loss: 1.8031 - val_accuracy: 0.7386
Epoch 2/10
90/90 [==============================] - 13s 140ms/step - loss: 0.1673 - accuracy: 0.9387 - val_loss: 2.2950 - val_accuracy: 0.7284
Epoch 3/10
90/90 [==============================] - 12s 136ms/step - loss: 0.1649 - accuracy: 0.9348 - val_loss: 2.1749 - val_accuracy: 0.7462
Epoch 4/10
90/90 [==============================] - 13s 138ms/step - loss: 0.1900 - accuracy: 0.9209 - val_loss: 1.8376 - val_accuracy: 0.7386
Epoch 5/10
86/90 [===========================>..] - ETA: 0s - loss: 0.1891 - accuracy: 0.9245

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',input_shape=(256,256,3),activation='relu')) 
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3),activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization()) 

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35)) #64 --> 42

model.add(Conv2D(64, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Flatten()) 
model.add(Dropout(0.5)) 
model.add(Dense(16,activation='relu')) 
model.add(BatchNormalization())
model.add(Dense(4,activation='softmax')) 

model.summary()

In [ ]:
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

In [ ]:
model.fit(train_data,validation_data=test_data, epochs =8)